In [12]:
from datasets import load_dataset
import pandas as pd
from utils import *

# Load SST dataset
dataset = load_dataset("sst")

train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])


## Preprocessing

In [13]:
train_df['sentiment'] = train_df['label'].apply(map_classes)  
test_df['sentiment'] = test_df['label'].apply(map_classes)
test_df  

,sentence,label,tokens,tree,sentiment
0,Effective but too-tepid biopic,0.513890,Effective|but|too-tepid|biopic,6|6|5|5|7|7|0,2
1,If you sometimes like to go to the movies to h...,0.736110,If|you|sometimes|like|to|go|to|the|movies|to|h...,40|39|38|37|36|34|33|32|32|31|30|30|29|28|26|2...,3
2,"Emerges as something rare , an issue movie tha...",0.861110,"Emerges|as|something|rare|,|an|issue|movie|tha...",42|41|40|40|43|38|37|37|36|34|31|31|32|30|30|2...,4
3,The film provides some great insight into the ...,0.597220,The|film|provides|some|great|insight|into|the|...,50|50|47|46|45|45|44|42|41|41|40|36|36|37|38|3...,2
4,Offers that rare combination of entertainment ...,0.833330,Offers|that|rare|combination|of|entertainment|...,16|14|13|13|12|10|10|11|17|11|12|15|14|15|16|17|0,4
...,...,...,...,...,...
2205,An imaginative comedy\/thriller .,0.777780,An|imaginative|comedy\/thriller|.,7|6|5|5|6|7|0,3
2206,"( A ) rare , beautiful film .",0.916670,"(|A|)|rare|,|beautiful|film|.",13|12|12|11|10|9|9|15|10|11|14|13|14|15|0,4
2207,( An ) hilarious romantic comedy .,0.888890,(|An|)|hilarious|romantic|comedy|.,12|11|11|9|8|8|10|9|10|13|12|13|0,4
2208,Never ( sinks ) into exploitation .,0.625000,Never|(|sinks|)|into|exploitation|.,11|10|9|9|8|8|13|12|10|11|12|13|0,3
